Importing libraries needed for analysis

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis


import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


Scrapping data for london locations from wikipedia page

In [ ]:
import wikipedia as wp
 
#Get the html source
html = wp.page("List of areas of London").html().encode("UTF-8")
df = pd.read_html(html)[1]
df = df.T.set_index(0).T
df.head()

In [9]:
df.columns = ['Location','London_borough','Post town','Postcode district','Dial Code','OS grid ref']

In [10]:
df['London_borough'] =  df['London_borough'].apply(lambda x: x.replace('[','').replace(']','')) 
df['London_borough'] =  df['London_borough'].str.replace('\d+', '')
df['London_borough'] =  df['London_borough'].str.split(',').str[0]
df['Postcode district'] =  df['Postcode district'].str.split(',').str[0]
df['Postcode district'] =  df['Postcode district'].str.split('(').str[0]
df['Post town'] =  df['Post town'].str.split(',').str[0]

In [107]:
df.shape

(532, 5)

In [109]:
df.head()

,Location,London_borough,Postcode district,Latitude,Longitude
1,Abbey Wood,Bexley,SE2,51.513240,-0.267460
2,Acton,Ealing,W3,51.384755,-0.051499
3,Addington,Croydon,CR0,51.384755,-0.051499
4,Addiscombe,Croydon,CR0,51.506420,-0.127210
5,Albany Park,Bexley,DA5,51.506420,-0.127210


Fetching Latitude and Longitude data

In [26]:
def getLatLong(row):
    #print('post :{}'.format(row[:]))
    #print('neigh :{}'.format(row[1]))
    # initialize your variable to None
    lat_lng_coords = None
    search_query = '{}, London,UK'.format(row)
    # loop until you get the coordinates
    try:
        while(lat_lng_coords is None):
            #g = geocoder.here(search_query,app_id=app_id,app_code=app_code)
            g = geocoder.arcgis(search_query)
            lat_lng_coords = g.latlng
            #print('FIRST')
    except IndexError:
        latitude = 0.0
        longitude = 0.0
        print('BACKUP')
        return [latitude,longitude]

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(latitude, longitude)
    return [latitude, longitude]

In [27]:
coords_list = df['Postcode district'].apply(getLatLong).tolist()

51.492450000000076 0.12127000000003818
51.51324000000005 -0.2674599999999714
51.38475500000004 -0.051498623999975734
51.38475500000004 -0.051498623999975734
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.51200000000006 -0.08057999999994081
51.51651000000004 -0.11967999999995982
51.50642000000005 -0.1272099999999341
51.41009000000008 -0.05682999999993399
51.523610000000076 -0.09876999999994496
51.27172802817503 0.07471050704375404
51.56393000000003 -0.12944999999996298
51.58370000000008 0.17476000000004888
51.64441500000004 -0.17918542799992565
51.616310000000055 -0.1383899999999585
51.44822000000005 -0.1483899999999494
51.499960000000044 -0.09567999999995891
51.523610000000076 -0.09876999999994496
51.53312000000005 0.08407653200004006
51.50642000000005 -0.1272099999999341
51.47052000000008 0.14670637700004363
51.47457000000003 -0.24211999999994305
51.452067760578885 0.1722299153689439
51.615680000000054 -0.2451099999999542
51.64441500000004 -0.1791854279

51.589270000000056 -0.16394999999994297
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.35142941071637 -0.24988108535584375
51.50878000000006 -0.3362999999999374
51.44864734579377 -0.3294047593453292
51.50642000000005 -0.1272099999999341
51.53916000000004 -0.2512299999999641
51.50510549999999 -0.4275742499999865
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.56878000000006 -0.09846999999996342
51.567890615208455 -0.3394832478241024
51.567890615208455 -0.3394832478241024
51.50642000000005 -0.1272099999999341
51.61291065511444 -0.37696639802957227
51.448444494776844 -0.44849227524521545
51.66208623796546 0.18662121631421688
51.50642000000005 -0.1272099999999341
51.50510549999999 -0.4275742499999865
51.50642000000005 -0.1272099999999341
51.58912000000004 -0.2280999999999267
51.45529000000005 -0.09927999999996473
51.50642000000005 -0.1272099999999341
51

51.50642000000005 -0.1272099999999341
51.50642000000005 -0.1272099999999341
51.45066000000003 -0.18996999999995978
51.63429000000008 -0.13365999999996347
51.52022000000005 -0.054309999999929914
51.392201353000075 -0.2159499999999639
51.49714000000006 -0.13828999999992675
51.45374249766627 -0.32996134872866295
51.51651000000004 -0.11967999999995982
51.462680000000034 -0.035579999999924894
51.53398000000004 -0.17377999999996518
51.523610000000076 -0.09876999999994496
51.50642000000005 -0.1272099999999341
51.524500000000046 -0.12272999999993317
51.50642000000005 -0.1272099999999341
51.58151000000004 -0.07589999999993324
51.62043771181625 -0.30130161650491155
51.52022000000005 -0.054309999999929914
51.476230000000044 -0.13055999999994583
51.56350000000003 -0.07312999999993508
51.53916000000004 -0.2512299999999641
51.540140000000065 0.0027800000000297587
51.45374249766627 -0.32996134872866295
51.41870000000006 -0.12752999999997883
51.56878000000006 -0.09846999999996342
51.50642000000005 -0.

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


51.58170000000007 -0.12092999999993026
51.45374249766627 -0.32996134872866295
51.54307422088052 0.25011357808526574
51.50642000000005 -0.1272099999999341
51.558970000000045 -0.05322999999992817
51.56393000000003 -0.12944999999996298
51.41990000000004 -0.08807999999993399
51.452067760578885 0.1722299153689439
51.58477000000005 -0.019429999999942993
51.50642000000005 -0.1272099999999341
51.53292000000005 0.05461000000002514
51.50642000000005 -0.1272099999999341
51.476230000000044 -0.13055999999994583
51.38475500000004 -0.051498623999975734
51.37538082912862 -0.15232342204289587
51.58477000000005 -0.019429999999942993
51.58477000000005 -0.019429999999942993
51.487640000000056 -0.09541999999993322
51.45066000000003 -0.18996999999995978
51.576760000000036 0.027230000000031396
51.52022000000005 -0.054309999999929914
51.50642000000005 -0.1272099999999341
51.444630000000075 0.05212000000005901
51.45907500000004 0.0991206960000568
51.50642000000005 -0.1272099999999341
51.55076000268121 -0.29333

In [28]:
df_coords = pd.DataFrame(coords_list, columns=['Latitude', 'Longitude'])

In [29]:
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [30]:
df.head()

,Location,London_borough,Postcode district,Latitude,Longitude
1,Abbey Wood,Bexley,SE2,51.513240,-0.267460
2,Acton,Ealing,W3,51.384755,-0.051499
3,Addington,Croydon,CR0,51.384755,-0.051499
4,Addiscombe,Croydon,CR0,51.506420,-0.127210
5,Albany Park,Bexley,DA5,51.506420,-0.127210


In [38]:
df = df.dropna()

In [93]:
df.shape

(532, 5)

In [31]:
# get the coordinates of Kuala Lumpur
address = 'London,UK'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London,UK {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London,UK 51.5073219, -0.1276474.


In [50]:
# create map of Toronto using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Location']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_london)  
    
map_london

In [51]:
map_london.save('map_kl.html')

In [52]:
CLIENT_ID = '3YL42BHE4U14EFRRFJTTLJ0RPQBRUDOWC5XJUCABYV5IDIRJ' # your Foursquare ID
CLIENT_SECRET = '1YOQMWPXYAXJNGZBAI0PQRESEWNI2MADFEOKLO4SPUHP5TOB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3YL42BHE4U14EFRRFJTTLJ0RPQBRUDOWC5XJUCABYV5IDIRJ
CLIENT_SECRET:1YOQMWPXYAXJNGZBAI0PQRESEWNI2MADFEOKLO4SPUHP5TOB


In [53]:
def getBuiltUrl(neigh_lat,neigh_long,radius=1400):
    # type your answer here
    LIMIT=100
    #radius=1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url

In [56]:
neigh_name, neigh_borough, neigh_post, neigh_lat, neigh_long = df.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))
results = requests.get(getBuiltUrl(neigh_lat,neigh_long)).json()
results

Latitude and longitude values of Abbey Wood are 51.51324000000005, -0.2674599999999714.


{'meta': {'code': 200, 'requestId': '5ec02b80edbcad001bf27185'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4cb84e45a33bb1f781d87ffd-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d11b941735',
         'name': 'Pub',
         'pluralName': 'Pubs',
         'primary': True,
         'shortName': 'Pub'}],
       'id': '4cb84e45a33bb1f781d87ffd',
       'location': {'address': 'Station Buildings',
        'cc': 'GB',
        'city': 'Acton',
        'country': 'United Kingdom',
        'crossStreet': 'Churchfield Rd.',
        'distance': 572,
        'formattedAddress': ['Station Buildings (Churchfield Rd.)',
         'Acton',
         'Greater London',
         'W3 6BH',
         'United Kin

In [57]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Fetching neighborhood information about different category of venues

In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Station House,Pub,51.508877,-0.263076
1,Bake Me,Creperie,51.508452,-0.268543
2,London Star Hotel,Hotel,51.509624,-0.272456
3,Acton Centre,Gym / Fitness Center,51.506608,-0.266878
4,Acton Park,Park,51.508595,-0.261573


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

80 venues were returned by Foursquare.


In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        LIMIT=100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
london_venues = getNearbyVenues(names=df['Location'], 
                                   latitudes=df['Latitude'], 
                                   longitudes=df['Longitude'], 
                                   radius=500)

Abbey Wood
Acton
Addington
Addiscombe
Albany Park
Aldborough Hatch
Aldgate
Aldwych
Alperton
Anerley
Angel
Aperfield
Archway
Ardleigh Green
Arkley
Arnos Grove
Balham
Bankside
Barbican
Barking
Barkingside
Barnehurst
Barnes
Barnes Cray
Barnet Gate
Barnet (also Chipping Barnet, High Barnet)
Barnsbury
Battersea
Bayswater
Beckenham
Beckton
Becontree
Becontree Heath
Beddington
Bedford Park
Belgravia
Bellingham
Belmont
Belmont
Belsize Park
Belvedere
Bermondsey
Berrylands
Bethnal Green
Bexley (also Old Bexley, Bexley Village)
Bexleyheath (also Bexley New Town)
Bickley
Biggin Hill
Blackfen
Blackfriars
Blackheath
Blackheath Royal Standard
Blackwall
Blendon
Bloomsbury
Botany Bay
Bounds Green
Bow
Bowes Park
Brentford
Brent Cross
Brent Park
Brimsdown
Brixton
Brockley
Bromley
Bromley (also Bromley-by-Bow)
Bromley Common
Brompton
Brondesbury
Brunswick Park
Bulls Cross
Burnt Oak
Burroughs, The
Camberwell
Cambridge Heath
Camden Town
Canary Wharf
Cann Hall
Canning Town
Canonbury
Carshalton
Castelnau
Cast

In [ ]:
london_venues.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [64]:
print(london_venues.shape)
london_venues.head()

(18471, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abbey Wood,51.51324,-0.26746,Sainsbury's Local,51.514967,-0.268977,Grocery Store
1,Abbey Wood,51.51324,-0.26746,Acton Main Line Railway Station (AML),51.517077,-0.267317,Train Station
2,Abbey Wood,51.51324,-0.26746,Co-op Food,51.515919,-0.268030,Grocery Store
3,Abbey Wood,51.51324,-0.26746,London Guest House,51.513973,-0.269680,Bed & Breakfast
4,Abbey Wood,51.51324,-0.26746,The Balti House,51.516627,-0.267307,Indian Restaurant


In [66]:
london_venues.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abbey Wood,7,7,7,7,7,7
Acton,2,2,2,2,2,2
Addington,2,2,2,2,2,2
Addiscombe,66,66,66,66,66,66
Albany Park,66,66,66,66,66,66
Aldborough Hatch,84,84,84,84,84,84
Aldgate,86,86,86,86,86,86
Aldwych,66,66,66,66,66,66
Alperton,6,6,6,6,6,6


In [67]:
print('There are {} uniques categories.'.format(len(london_venues['VenueCategory'].unique())))

There are 320 uniques categories.


In [68]:
london_venues['VenueCategory'].unique()[:50]

array(['Grocery Store', 'Train Station', 'Bed & Breakfast',
       'Indian Restaurant', 'Park', 'Breakfast Spot',
       'Construction & Landscaping', 'Plaza', 'Fountain', 'Hotel',
       'Monument / Landmark', 'Art Museum', 'Art Gallery', 'Garden',
       'Wine Bar', 'Thai Restaurant', 'Indie Movie Theater', 'Pub',
       'North Indian Restaurant', 'Burger Joint', 'Theater',
       'Opera House', 'Sandwich Place', 'Currywurst Joint',
       'French Restaurant', 'Ice Cream Shop', 'Boutique', 'Spa',
       'Japanese Restaurant', 'Liquor Store', 'Pharmacy',
       'Greek Restaurant', 'Bar', 'Tour Provider', 'Bakery',
       'English Restaurant', 'Lounge', 'Cocktail Bar', 'Restaurant',
       'Pakistani Restaurant', 'Tea Room', 'Ramen Restaurant', 'Café',
       'Lebanese Restaurant', 'Chinese Restaurant', 'Italian Restaurant',
       'Steakhouse', 'Outdoor Sculpture', 'Salad Place', 'Scenic Lookout'],
      dtype=object)

In [71]:
"Shopping Mall" in london_venues['VenueCategory'].unique()

True

In [78]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhoods'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

print(london_onehot.shape)
london_onehot.head()

(18471, 321)


,Neighborhoods,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Casino,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Racetrack,Radio Station,Ramen Restaurant,Real Estate Office,Record Shop,Recording Studio,Recreation Center,Recruiting Agency,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,

In [79]:
london_grouped = london_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(london_grouped.shape)
london_grouped.head()

(522, 321)


,Neighborhoods,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Casino,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Racetrack,Radio Station,Ramen Restaurant,Real Estate Office,Record Shop,Recording Studio,Recreation Center,Recruiting Agency,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,

In [80]:
len(london_grouped[london_grouped["Shopping Mall"] > 0])

12

In [81]:
london_mall = london_grouped[["Neighborhoods","Shopping Mall"]]

In [82]:
london_mall.head()

,Neighborhoods,Shopping Mall
0,Abbey Wood,0.0
1,Acton,0.0
2,Addington,0.0
3,Addiscombe,0.0
4,Albany Park,0.0


In [83]:
# set number of clusters
kclusters = 3

kl_clustering = london_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [84]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
london_merged = london_mall.copy()

# add clustering labels
london_merged["Cluster Labels"] = kmeans.labels_

In [86]:
london_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
london_merged

,Neighborhood,Shopping Mall,Cluster Labels
0,Abbey Wood,0.000000,0
1,Acton,0.000000,0
2,Addington,0.000000,0
3,Addiscombe,0.000000,0
4,Albany Park,0.000000,0
5,Aldborough Hatch,0.000000,0
6,Aldgate,0.000000,0
7,Aldwych,0.000000,0
8,Alperton,0.000000,0
9,Anerley,0.000000,0


In [91]:
london_df = df.copy()

In [92]:
london_df.shape

(532, 5)

In [95]:
london_df = london_df.rename(columns = {'Location' : 'Neighborhood'})

In [97]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(london_df.set_index("Neighborhood"), on="Neighborhood")

print(london_merged.shape)
london_merged.head()

(528, 7)


,Neighborhood,Shopping Mall,Cluster Labels,London_borough,Postcode district,Latitude,Longitude
0,Abbey Wood,0.0,0,Bexley,SE2,51.513240,-0.267460
1,Acton,0.0,0,Ealing,W3,51.384755,-0.051499
2,Addington,0.0,0,Croydon,CR0,51.384755,-0.051499
3,Addiscombe,0.0,0,Croydon,CR0,51.506420,-0.127210
4,Albany Park,0.0,0,Bexley,DA5,51.506420,-0.127210


In [98]:
# sort the results by Cluster Labels
print(london_merged.shape)
london_merged.sort_values(["Cluster Labels"], inplace=True)
london_merged

(528, 7)


,Neighborhood,Shopping Mall,Cluster Labels,London_borough,Postcode district,Latitude,Longitude
0,Abbey Wood,0.000000,0,Bexley,SE2,51.513240,-0.267460
352,Penge,0.000000,0,Bromley,SE20,51.529690,-0.086970
351,Peckham,0.000000,0,Southwark,SE15,51.410090,-0.056830
349,Park Royal,0.000000,0,Brent,NW10,51.477720,-0.201450
348,Palmers Green,0.000000,0,Enfield,N13,51.539160,-0.251230
347,Paddington,0.000000,0,Westminster,W2,51.620030,-0.106600
346,Oval,0.000000,0,Lambeth,SW8,51.514940,-0.180480
345,Osterley,0.000000,0,Hounslow,TW7,51.476230,-0.130560
344,Osidge,0.000000,0,Barnet,N14,51.474389,-0.343965
343,Orpington,0.000000,0,Bromley,BR5,51.634290,-0.133660


In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [100]:
map_clusters.save('map_clusters.html')

In [103]:
london_merged.loc[london_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,London_borough,Postcode district,Latitude,Longitude
0,Abbey Wood,0.0,0,Bexley,SE2,51.513240,-0.267460
352,Penge,0.0,0,Bromley,SE20,51.529690,-0.086970
351,Peckham,0.0,0,Southwark,SE15,51.410090,-0.056830
349,Park Royal,0.0,0,Brent,NW10,51.477720,-0.201450
348,Palmers Green,0.0,0,Enfield,N13,51.539160,-0.251230
347,Paddington,0.0,0,Westminster,W2,51.620030,-0.106600
346,Oval,0.0,0,Lambeth,SW8,51.514940,-0.180480
345,Osterley,0.0,0,Hounslow,TW7,51.476230,-0.130560
344,Osidge,0.0,0,Barnet,N14,51.474389,-0.343965
343,Orpington,0.0,0,Bromley,BR5,51.634290,-0.133660


In [104]:
london_merged.loc[london_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,London_borough,Postcode district,Latitude,Longitude
369,Queensbury,0.081081,1,Harrow,HA3,51.579509,0.188665
488,Wembley Park,0.081081,1,Brent,HA9,51.579509,0.188665
404,South Harrow,0.081081,1,Harrow,HA2,51.579509,0.188665


In [106]:
london_merged.loc[london_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,London_borough,Postcode district,Latitude,Longitude
433,Stonebridge,0.018868,2,Brent,NW10,51.54014,0.00278
52,Blendon,0.022727,2,Bexley,DA 5,51.52450,-0.12273
425,St Mary Cray,0.022727,2,Bromley,BR5,51.52450,-0.12273
136,Dulwich,0.011236,2,Southwark,SE21,51.51406,-0.30073
291,Marks Gate,0.018868,2,Barking and Dagenham,RM6,51.54014,0.00278
259,Kilburn,0.022727,2,Brent,NW6,51.52450,-0.12273
233,Hither Green,0.022727,2,Lewisham,SE13,51.52450,-0.12273
350,Parsons Green,0.018182,2,Hammersmith and Fulham,SW6,51.47218,-0.06779
336,Notting Hill,0.018182,2,Kensington and Chelsea,W11,51.47218,-0.06779


Observations:

Most of the shopping malls are concentrated in the few area of London city, with the highest number in **cluster 1** and moderate number in cluster 2. On the other hand, **cluster 0** has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in **cluster 1** are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in **cluster 0** with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in **cluster 2** with moderate competition. Lastly, property developers are advised to avoid neighborhoods in **cluster 1** which already have high concentration of shopping malls and suffering from intense competition.
